In [1]:
import torch
print(torch.__version__)

2.7.1+cpu


In [2]:
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.metrics import classification_report
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
import nltk
from scipy.special import softmax

In [3]:
import sys
print(sys.executable)


C:\ProgramData\anaconda3\python.exe


In [4]:
DATASET_COLUMNS = ['target', 'ids', 'date', 'flag', 'user', 'text']
DATASET_ENCODING = "ISO-8859-1"
data = pd.read_csv('sentiment140.csv', encoding=DATASET_ENCODING, names=DATASET_COLUMNS)

In [5]:
data

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [6]:
print("Unique value counts in the target column:")
print(data['target'].value_counts())

Unique value counts in the target column:
target
0    800000
4    800000
Name: count, dtype: int64


In [7]:
def clean_text(text):
    stopwordlist = [
        'a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
        'and', 'any', 'are', 'as', 'at', 'be', 'because', 'been', 'before',
        'being', 'below', 'between', 'both', 'by', 'can', 'd', 'did', 'do',
        'does', 'doing', 'down', 'during', 'each', 'few', 'for', 'from',
        'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
        'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
        'into', 'is', 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma',
        'me', 'more', 'most', 'my', 'myself', 'needn', 'no', 'nor', 'now',
        'o', 'of', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves',
        'out', 'own', 're', 's', 'same', 'she', "shes", 'should', "shouldve", 'so', 'some', 'such',
        't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
        'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
        'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was',
        'we', 'were', 'what', 'when', 'where', 'which', 'while', 'who', 'whom',
        'why', 'will', 'with', 'won', 'y', 'you', "youd", "youll", "youre",
        "youve", 'your', 'yours', 'yourself', 'yourselves'
    ]

    
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return wordnet.NOUN
    
   
    text = text.lower()
    text = re.sub(r'((www\.[^\s]+)|(https?://[^\s]+))', ' ', text)
    text = re.sub(r'@[\S]+', 'USER', text)
    text = re.sub(r'#(\S+)', r'\1', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    text = " ".join([word for word in text.split() if word not in stopwordlist])
    tokenizer = RegexpTokenizer(r'\w+|[^\w\s]')
    tokens = tokenizer.tokenize(text)

 
    pos_tags = nltk.pos_tag(tokens)

    
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in pos_tags]
    
    return " ".join(lemmatized_tokens)

In [10]:
import re

def clean_text(text):
    text = re.sub(r"http\S+", "", text)      
    text = re.sub(r"@\w+", "USER", text)     
    text = re.sub(r"[^a-zA-Z\s]", "", text)  
    text = text.lower()                      
    return text

data['text'] = data['text'].apply(clean_text)


In [11]:
positive_samples = data[data['target'] == 4].sample(500, random_state=42)
negative_samples = data[data['target'] == 0].sample(500, random_state=42)
data_subset = pd.concat([positive_samples, negative_samples])

data_subset['target'] = data_subset['target'].replace(4, 1)

In [14]:
task = 'sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


def predict_sentiment(text):
    encoded_input = tokenizer(text, return_tensors='pt').to(device)
    with torch.no_grad():
        output = model(**encoded_input)
    scores = output[0][0].cpu().numpy()
    scores = softmax(scores)
   
    negative_score = scores[0]
    positive_score = scores[2]
    return 1 if positive_score > negative_score else 0


data_subset['sentiment'] = data_subset['text'].apply(predict_sentiment)


In [15]:
y_true = data_subset['target']
y_pred = data_subset['sentiment']
report = classification_report(y_true, y_pred, target_names=["negative", "positive"])

print(report)

              precision    recall  f1-score   support

    negative       0.79      0.74      0.76       500
    positive       0.75      0.80      0.78       500

    accuracy                           0.77      1000
   macro avg       0.77      0.77      0.77      1000
weighted avg       0.77      0.77      0.77      1000



In [16]:
def test_single_tweet(tweet):
    tweet = clean_text(tweet)
    sentiment = predict_sentiment(tweet)

   
    return "positive" if sentiment == 1 else "negative"

In [17]:
test_tweet = "I hate the website page i did not like it"
predicted_sentiment = test_single_tweet(test_tweet)
print(f"The sentiment of the tweet '{test_tweet}' is {predicted_sentiment}.")

The sentiment of the tweet 'I hate the website page i did not like it' is negative.
